## игра в шахматы 
(задания: 5, 6, 7 | суммарная сложность = 3)

In [1]:
from board import Board
Board().play()


       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  p  p  p  p  p  p  p  p  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  -  -  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  P  P  P  P  P  P  P  P  |  2 
 1  |  R  N  B  Q  K  B  N  R  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 2
выберите фигуру  : e2

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  p  p  p  p  p  p  p  p  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  -  -  -  -  -  |  5 
 4  |  -  -  -  - :-: -  -  -  |  4 
 3  |  -  -  -  - :-: -  -  -  |  3 
 2  |  P  P  P  P  P  P  P  P  |  2 
 1  |  R  N  B  Q  K  B  N  R  |  1 
      ------------------------     

0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 4
сколько ходов? 2

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  p  p  p  -  p  p  p  p  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  p  -  -  -  -  |  5 
 4  |  -  -  -  -  P  -  -  -  |  4 
 3  |  -  -  -  -  -  -  -  -  |  3 
 2  |  P  P  P  P  -  P  P  P  |  2 
 1  |  R  N  B  Q  K  B  N  R  |  1 
      ------------------------      
       A  B  C  D  E  F  G  H       

-- white --
0. закончить игру
1. сделать ход
2. куда можно сходить фигурой
3. мои фигуры под ударом
4. откат хода/ходов
? : 1
-- ход № 3 --
выберите фигуру  : b1
куда переместить : c3

       A  B  C  D  E  F  G  H       
      ------------------------      
 8  |  r  n  b  q  k  b  n  r  |  8 
 7  |  p  p  p  -  p  p  p  p  |  7 
 6  |  -  -  -  -  -  -  -  -  |  6 
 5  |  -  -  -  p  -  -  -  -  |  5 
 4  |  -  -  -  

In [2]:
from board import Board
from pieces import Piece
import numpy as np
from constants import WHITE, BLACK

In [3]:


class Checker(Piece):
    def __init__(self, color, x, y):
        # как выглядит на доске
        self.image = ' H ' if color == WHITE else ' h '
        # цвет фигуры
        self.color = color
        # позиция на доске
        self.position = (x, y)


    def move(self, to_x, to_y, locations, killers):
        # перемещение, смена координат
        move_lst = self.get_moves_list(locations[0], locations[1])
        print(f'move_lst {move_lst}')
        if move_lst and (to_x, to_y) in move_lst: # изменяем координаты фигуры на новые
            killing_lst = self.is_killing(locations[0], locations[1], to_x, to_y)
            if not killing_lst and killers:  # если выбранная фигура просто перемещается, а другие могут съесть
                return False, []  # то такой ход делать нельзы
            self.position = (to_x, to_y)
            return True, killing_lst  # переместить получилось, список съедаемых фигур
        else:
            return False, []
        
          
    def get_moves_dict(self, white_locations, black_locations):
        # получить список словарей положений, куда фигура может сходить и флаг - съедаем или нет фигуру
        moves_list = []
        killing_moves_list = []
        if self.color == WHITE:
            targets = [(1, 1), (-1, 1)]
            enemies_list = black_locations
            friends_list = white_locations
        else:
            targets = [(-1, -1), (1, -1)]
            friends_list = black_locations
            enemies_list = white_locations
            
        # куда можно сходить, никого не съедая
        for target in targets:
            x, y = target
            if (self.position[0] + x,  self.position[1] + y) not in friends_list + enemies_list and \
                        0 <=  self.position[0] + x <= 7 and 0 <=  self.position[1] + y <= 7:
                    moves_list.append(( self.position[0] + x,  self.position[1] + y))
            
        # куда можно сходить, съедая
        def make_chain(killing_moves_list, tmp_pos, target, d):  # рассмотрим все возможные цепочки с поеданием
            x, y = target
            targets = [(1, -1), (-1, -1), (1, 1), (-1, 1)]
            if ( tmp_pos[0] + x,  tmp_pos[1] +  y) in enemies_list and \
                    0 <= tmp_pos[0] +  x <= 7 and 0 <=  tmp_pos[1] + y <= 7 \
                    and (( tmp_pos[0] + x * 2,  tmp_pos[1] + y * 2) not in friends_list + enemies_list \
                    or ( tmp_pos[0] + x * 2,  tmp_pos[1] + y * 2) == tmp_pos) and \
                    0 <=  tmp_pos[0] + x * 2 <= 7 and 0 <=  tmp_pos[1] + y * 2 <= 7 :
                # если мы можем передвинуться на новую клетку
                tmp_pos = ( tmp_pos[0] + x * 2,  tmp_pos[1] + y * 2)  # изменяем воображаемую позицию фигуры
                d[tmp_pos] = ( tmp_pos[0] - x,  tmp_pos[1] - y)  # добавляем это действие в словарь данного пути
                killing_moves_list.append(d)  # добавляем словарь в список ходов
                if tmp_pos != self.position:  # если мы не вернулись на исходную позицию фигуры
                    for i in targets:
                        if not(i[0] == target[0]*(-1) and i[1] == target[1]*(-1)):
                             # продолжаем искать пути, исключая направление, откуда пришли
                            make_chain(killing_moves_list, tmp_pos , i, d.copy()) 
            return None
        
        my_killing_moves_list = []
        make_chain(my_killing_moves_list, self.position, (1, -1),{})
        make_chain(my_killing_moves_list, self.position, (-1, -1), {})
        make_chain(my_killing_moves_list, self.position, (1, 1), {})
        make_chain(my_killing_moves_list, self.position, (-1, 1), {})
        
        # удаляем варианты ходов, которые являются частью более большого хода
        list_copy = my_killing_moves_list.copy()
        print(list_copy)
        for dict1 in my_killing_moves_list:
            for dict2 in my_killing_moves_list:
                if dict1 != dict2 and all(item in dict2.items() for item in dict1.items()):
                    if dict1 in list_copy:
                        list_copy.remove(dict1)
        if list_copy:
            # если есть возможность, то обязаны съесть, возвращаем только ходы, где съедается
            print(f'moves_dict   {list_copy}')
            return list_copy, True  # True - съедаем
        return [{element: None} for element in moves_list], False  # False - не съедаем, преобразовываем к спискку словаря

    def get_moves_list(self, white_locations, black_locations):
        # получаем список позиций, куда фигура может переместиться
        moves_lst_dict, flag = self.get_moves_dict(white_locations, black_locations)
        return [list(dict_items.keys())[-1] for dict_items in moves_lst_dict]
    
    def is_killing(self, white_locations, black_locations, to_x, to_y):
        # возвращаем список съедаемых фигур
        moves_lst_dict, flag = self.get_moves_dict(white_locations, black_locations)
        print(moves_lst_dict, flag)
        killed_list = []
        if flag:  # создаем список съедаемых фигур, если такие есть
            for dictionary in moves_lst_dict:
                if list(dictionary.keys())[-1] == (to_x, to_y):
                    killed_list = list(dictionary.values())
                    break
        print(f'is_killing   {killed_list}')
        return killed_list



In [4]:
c = Checker(WHITE, 2, 3)



In [5]:
class CheckerQueen(Piece):
    def __init__(self, color, x, y):
        super().__init__(color, x, y)
        # как выглядит на доске
        self.image = ' D ' if color == WHITE else ' d '
        
    def get_moves_list(self, white_locations, black_locations):
        pass

In [ ]:

### - наследуем от Board


class CheckersBoard(Board):
    def __init__(self):
        # номер хода
        self.count = 0
        # запись истории партии
        self.history = None # история для шашаек не реализуется
        # чей сейчас ход
        self.turn = WHITE  # WHITE или BLACK

        # белые фигуры     
        self.white_pieces_list = [
                                    Checker(WHITE, 0, 5),
                                    Checker(WHITE, 2, 5),
                                    Checker(WHITE, 4, 5),
                                    Checker(WHITE, 6, 5),
                                    Checker(WHITE, 1, 6),
                                    Checker(WHITE, 3, 6),
                                    Checker(WHITE, 5, 6),
                                    Checker(WHITE, 7, 6),
                                    Checker(WHITE, 0, 7),
                                    Checker(WHITE, 2, 7),
                                    Checker(WHITE, 4, 7),
                                    Checker(WHITE, 6, 7)
                                ]
        # черные фигуры 
        self.black_pieces_list = [
                                    
                                    Checker(BLACK, 1, 0),
                                    Checker(BLACK, 3, 0),
                                    Checker(BLACK, 5, 0),
                                    Checker(BLACK, 7, 0),
                                    Checker(BLACK, 0, 1),
                                    Checker(BLACK, 2, 1),
                                    Checker(BLACK, 4, 1),
                                    Checker(BLACK, 6, 1),
                                    Checker(BLACK, 1, 2),
                                    Checker(BLACK, 3, 2),
                                    Checker(BLACK, 5, 2),
                                    Checker(BLACK, 7, 2)
                                ]

### def get_piece(self, coords, color):  


### def get_white_locations(self):  

### def get_black_locations(self):

### def to_int_coords(self, xy: str):        
  
    def create_piece(self, color, pos):
        # создаем и возвращаем дамку
        return CheckerQueen(color, pos[0], pos[1])
        
    
    def moves_back(self, n_moves: int):
        # метод 'сделать ходы назад' не реализуется
        pass

        
  ###  def is_in_danger(self):
    

  ###  def print_board(self, in_danger=False, coords=None): 

    def get_killers(self):
        # список фигур, которые могут съесть фигуру противника в данном ходу
        lst = []
        if self.turn == WHITE:
            pieces_list = self.white_pieces_list
        else:
            pieces_list = self.black_pieces_list
            
        for p in pieces_list:
            moves_lst_dict, flag = p.get_moves_dict(self.get_white_locations(), self.get_black_locations())
            if flag:  # если у фигуры ходы с поеданием
                lst.append(p)
        return lst
            
    def move_piece(self, from_coords, to_coords):
        # сделать ход, записать в историю (если фигура съедена, то удалить ее), кол-во ходов в партии +1
        # ход получилось сделать - True, нельзя сделать такой ход - False
        if self.turn == WHITE:  # если сейчас ход белых
            wh_p = self.get_piece(from_coords, WHITE)  
            if wh_p:  # если выбранная фигура существует
                flag, killing_list = wh_p.move(to_coords[0], to_coords[1], (self.get_white_locations(), self.get_black_locations()), self.get_killers())  # если указан правильный ход
                print(flag, killing_list)
                if flag:
                    for bl_p_pos in killing_list:
                        bl_p = self.get_piece(bl_p_pos, BLACK) 
                        print(bl_p )
                        print(bl_p.position )
                        self.black_pieces_list.remove(bl_p)
                    # передаем ход черным, кол-во ходов в партии +1
                    self.turn = BLACK
                    self.count += 1
                    
                    
                    # проыерка на дамки -> превращение в дамки
                    
                    
                    return True

        else:  # если сейчас ход черных (аналогично белым)
            bl_p = self.get_piece(from_coords, BLACK)
            if bl_p:
                flag, killing_list = bl_p.move(to_coords[0], to_coords[1], (self.get_white_locations(),self.get_black_locations()), self.get_killers())
                if flag:
                    for wh_p_pos in killing_list:
                        wh_p = self.get_piece(wh_p_pos, WHITE)
                        self.white_pieces_list.remove(wh_p)
                    self.turn = WHITE
                    self.count += 1
                    return True
                
        return False
    
    def is_in_danger(self):
        # возвращает список координат фигур ходящего игрока, которые находятся в опасности 
        options_list = []  # куда могут сходить все фигуры противника
        if self.turn == BLACK:
            # ходят черные, угроза от белых
            for p in self.white_pieces_list:
                for move in p.get_moves_list(self.get_white_locations(), self.get_black_locations()):
                    options_list.append(p.is_killing(self.get_white_locations(), self.get_black_locations(), move[0], move[1]))
            p_list = self.get_black_locations()
        else:
            # ходят белые, угроза от черных
            for p in self.black_pieces_list:
                for move in p.get_moves_list(self.get_white_locations(), self.get_black_locations()):
                    options_list.append(p.is_killing(self.get_white_locations(), self.get_black_locations(), move[0], move[1]))
            p_list = self.get_white_locations()
        
        in_danger_list = []
        # проверка, попадает ли фигура в хотя бы какой-то из списков возможных ходов фигур противника
        for i in range(len(p_list)):
            for option in options_list:
                if p_list[i] in option:
                    in_danger_list.append(p_list[i])
        return in_danger_list

    
    
# написали заново (просто убрали пункт 'откат ходов')
    def play(self):  
        # игра (меню действий и общение с пользователем)
        self.print_board()
        print(f'-- {self.turn} --')
        menue = '0. закончить игру\n1. сделать ход\n2. куда можно сходить фигурой\n3. мои фигуры под ударом\n? : '
        choice = input(menue)
        while choice != '0':
            match choice:
                case '1':  # делаем ход, неправильные координаты -> обратно в меню
                    #self.print_board()
                    print(f'-- ход № {self.count+1} --')
                    from_coords = self.to_int_coords(input('выберите фигуру  : '))
                    to_coords = self.to_int_coords(input('куда переместить : '))
                    if not(from_coords and to_coords and self.move_piece(from_coords, to_coords)):
                        print('\n ~ неправильные координаты ! ~ ')
                    self.print_board()

                case '2':  # возможные ходы выбранной фигуры
                    f_coords = self.to_int_coords(input('выберите фигуру  : '))
                    if f_coords:
                        lst, flag = self.get_piece(f_coords, self.turn).get_moves_dict(self.get_white_locations(), self.get_black_locations())
                        if not flag and self.get_killers():
                            self.print_board()  # мы не можем ходить этой фигурой, так как она не поедает, а другая может съесть
                        else:
                            self.print_board(coords=f_coords)
                    else:
                        print('\n ~ неправильные координаты ! ~ ')
                        self.print_board()
                        
                case '3':  # фигуры под ударом
                    self.print_board(in_danger=True)
                    
            print(f'-- {self.turn} --')
            choice = input(menue)
            
        # заканчиваем игру
        choice_2 = input('\n1) сдаться\n2) ничья\n? ')
        if choice_2 == '2':  # если выбрали ничью
            print(f'----------------------------\n  ничья !')
        else:  # если выбрали сдаться или несуществующий вариант
            if self.turn != WHITE:
                print(f'----------------------------\n  белые выиграли !')
            else:
                print(f'----------------------------\n  черные выиграли !')



In [ ]:
c = CheckersBoard()
c.play()